Follows tutorial here:
https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

In [106]:
import pandas as pd
import numpy as np
import pickle
import re
from os import walk
from os import listdir
from os.path import isfile, join
import json

# Load Data

In [122]:
# a.split("\t")
data_path = '../data/amazon_reviews/amazon_review/'

In [123]:
def load_data(filepath):
    with open(filepath, "r") as file:
        text = file.read()
    reviews = text.split("\n")
    reviews = [i for i in reviews if (i != "") and ("UNCONFIDENT_INTENT_FROM_SLAD" not in i)]
    x = [x.split("\t")[0] for x in reviews]
    y = [x.split("\t")[1] for x in reviews]
    return x, y

In [128]:
set(load_data(data_path + "test/" + "Baby.test")[1])

{'1', '2'}

In [137]:
all_data = {}
for domain in ["train", "dev", "test"]:
    folder_path = data_path + domain + "/"
    onlyfiles = [f for f in listdir(folder_path) if isfile(join(folder_path, f))]
    for file in onlyfiles:
        if (".test" in file) or (".train" in file) or (".test" in file):
            all_data[file] = load_data(folder_path + "/" + file)

In [138]:
all_data.keys()

dict_keys(['CDs_and_Vinyl.train', 'Clothing_Shoes_and_Jewelry.train', 'Home_and_Kitchen.train', 'Beauty.train', 'Sports_and_Outdoors.train', 'Movies_and_TV.train', 'Apps_for_Android.train', 'Cell_Phones_and_Accessories.train', 'Electronics.train', 'Office_Products.train', 'Books.train', 'Health_and_Personal_Care.train', 'Kindle_Store.train', 'Grocery_and_Gourmet_Food.train', 'Pet_Supplies.train', 'Tools_and_Home_Improvement.test', 'Tools_and_Home_Improvement.train', 'Pet_Supplies.test', 'Automotive.test', 'Automotive.train', 'Grocery_and_Gourmet_Food.test', 'Baby.test', 'Video_Games.train', 'Baby.train', 'Digital_Music.test', 'Digital_Music.train', 'Toys_and_Games.train', 'Toys_and_Games.test', 'Video_Games.test'])

In [112]:
with open("../data/all_cleaned/amazon_data_dict.txt","w") as f:
    json.dump(all_data, f, indent=4)

# LDA

In [155]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.stem import *
import numpy as np
np.random.seed(2021)
import nltk
nltk.download('wordnet')
stemmer = SnowballStemmer("english")

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/yuchen.zhang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [168]:
freq_used = ['Home_and_Kitchen', 'Books', 'Electronics', 'Movies_and_TV']

In [157]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [166]:
all_data['CDs_and_Vinyl.train'][1][0]

'1'

In [158]:
preprocess(all_data['CDs_and_Vinyl.train'][0][0])

['perfect',
 'event',
 'want',
 'tape',
 'own',
 'tape',
 'quot',
 'quot',
 'want',
 'visual',
 'engag',
 'listen']

In [172]:
all_data.keys()

dict_keys(['CDs_and_Vinyl.train', 'Clothing_Shoes_and_Jewelry.train', 'Home_and_Kitchen.train', 'Beauty.train', 'Sports_and_Outdoors.train', 'Movies_and_TV.train', 'Apps_for_Android.train', 'Cell_Phones_and_Accessories.train', 'Electronics.train', 'Office_Products.train', 'Books.train', 'Health_and_Personal_Care.train', 'Kindle_Store.train', 'Grocery_and_Gourmet_Food.train', 'Pet_Supplies.train', 'Tools_and_Home_Improvement.test', 'Tools_and_Home_Improvement.train', 'Pet_Supplies.test', 'Automotive.test', 'Automotive.train', 'Grocery_and_Gourmet_Food.test', 'Baby.test', 'Video_Games.train', 'Baby.train', 'Digital_Music.test', 'Digital_Music.train', 'Toys_and_Games.train', 'Toys_and_Games.test', 'Video_Games.test'])

In [173]:
topics = {}
for i in freq_used:
    processed_train = [preprocess(x) for x in all_data[i + '.train'][0]]
    topics[i] = processed_train